### Scrapping Car prices data

## Import Libraries

In [39]:
import pandas as pd
import requests
import numpy as np
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')

# Links url

In [2]:
url="https://autos.mercadolibre.com.ar/auto_Desde_48_ITEM*CONDITION_2230581_NoIndex_True"
response=requests.get(url)
response

<Response [200]>

- response.text: gives you all HTML software of this URL in String. 

- response.content: gives you all HTML software of this URL in normal not string.

# Try to find the link detail info of each car in 1 page

In [3]:
soup = BeautifulSoup(response.content, 'html.parser')

In [4]:
webcam_divs = soup.find_all('div', {'class': 'ui-search-item__group ui-search-item__group--title shops__items-group'})

In [5]:
links=[]
for div in webcam_divs:
    link = div.find('a', {'class': 'ui-search-item__group__element shops__items-group-details ui-search-link'})
    href = link['href']
    links.append(href)
len(links)

48

# Scrapping all links

In [6]:
urls_all_pages = []
total_of_pages= 40
for i in range(total_of_pages):
    url=f"https://autos.mercadolibre.com.ar/auto_Desde_{i}_ITEM*CONDITION_2230581_NoIndex_True"
    
    response=requests.get(url)
    if response.status_code==200:
        soup = BeautifulSoup(response.content, 'html.parser')
        webcam_divs = soup.find_all('div', {'class': 'ui-search-item__group ui-search-item__group--title shops__items-group'})
        for div in webcam_divs:
            link = div.find('a', {'class': 'ui-search-item__group__element shops__items-group-details ui-search-link'})
            href = link['href']
            urls_all_pages.append(href)
    
    else:
        
        pass

In [7]:
len(urls_all_pages)

1920

### Back up all links into csv file

In [8]:
link_data=pd.DataFrame()
link_data["links"]=urls_all_pages
link_data.to_csv("links.csv",index=False)

**Craw car name**

In [41]:
import json
import re

urls_all_pages=pd.read_csv("links.csv")
urls_all_pages.iloc[0]['links']

'https://auto.mercadolibre.com.ar/MLA-1368741115-vw-volkswagen-nivus-2020-comfortline-200-tsi-at-abasto-motor-_JM#position=1&search_layout=grid&type=item&tracking_id=e2f455cd-7839-4077-b226-82e30c153c58'

In [3]:
response = requests.get(urls_all_pages.iloc[0]['links'])
soup = BeautifulSoup(response.content, 'html.parser')
car_name = soup.find('h1', {'class': 'ui-pdp-title'}).text.strip()
car_name

'Volkswagen Nivus 1.0 Tsi Tiptronic Comfortline'

In [4]:
# create a pandas DataFrame from the dictionary
df = pd.DataFrame({'Name': [car_name]})
df

,Name
0,Volkswagen Nivus 1.0 Tsi Tiptronic Comfortline


**Craw detail info (JSON)**

In [5]:
response=requests.get(urls_all_pages.iloc[1919]['links'])
content = response.text


melidata_list = re.findall(r'melidata\("add", "event_data", ({.*?})\);', content)

# convert each string to a dictionary
data = [json.loads(m) for m in melidata_list]

# create the dataframe
df2 = pd.DataFrame(data)
df2

,seller_id,seller_type,reputation_level,power_seller_status,price,currency_id,item_id,category_id,buying_mode,category_path,...,price_comparison_position,has_good_price,whatsapp_available,whatsapp_integration_available,map_item_attributes,new_wording_free,available_consumer_credit,has_seller_level_kyc,has_highlighted_sale_specs,verified
0,468445513,car_dealer,5_green,silver,3300000,ARS,MLA1412015300,MLA1744,classified,"[MLA1743, MLA1744]",...,0.92,False,True,False,"[{'id': 'BRAND', 'name': 'Marca', 'value_id': ...",True,False,True,False,False


# Start craw all cars

In [45]:
dataframes = []
# Run loop with multiple times likes 0->100, 100->200
for i in range(1000,1919):
    response=requests.get(urls_all_pages.iloc[i]['links'])
    content = response.text

    melidata_list = re.findall(r'melidata\("add", "event_data", ({.*?})\);', content)

    # convert each string to a dictionary
    data = [json.loads(m) for m in melidata_list]

    # create and append the dataframe
    df = pd.DataFrame(data)
    dataframes.append(df)
df_combined = pd.concat(dataframes, ignore_index=True)

In [46]:
df_combined

""


### Save in csv file

In [47]:
# df_combined.to_csv('data_crawed.csv')

In [48]:
import csv

with open('data_crawed.csv', 'a') as file:
    df_combined.to_csv(file, header=False, index=False)